In [1]:
#%pip install ras-commander pandas requests pathlib matplotlib

# 14_Core_Sensitivity.ipynb
Testing Core Sensitivity for RAS using the Bald Eagle Creek Multi-Gage 2D project.  


This should take around 15-45 minutes to run depending on your hardware.

In [ ]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation
try:
    # Try to import from the installed package
    from ras_commander import init_ras_project, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasHdf, ras
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import init_ras_project, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasHdf, ras

print("ras_commander imported successfully")

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ras_commander import RasExamples, init_ras_project, RasCmdr, RasPlan, RasGeo

# Step 1: Initialize RasExamples and extract the BaldEagleCrkMulti2D project
ras_examples = RasExamples()
ras_examples.extract_project(["BaldEagleCrkMulti2D"])

# Use Path.cwd() to get the current working directory in a Jupyter Notebook
current_directory = Path.cwd()
project_path = current_directory / "example_projects" / "BaldEagleCrkMulti2D"

# Step 2: Initialize the Muncie Project using init_ras_project (from ras_commander)
muncie_project = init_ras_project(project_path, "6.6")

# Step 3: Initialize a DataFrame to store execution results
results = []

# Step 4: Run sensitivity analysis for Plan 03 with core counts 1-8
plan_number = '03'
print(f"Running sensitivity analysis for Plan {plan_number}")

# Clear geompre files before running the plan
plan_path = RasPlan.get_plan_path(plan_number)
RasGeo.clear_geompre_files(plan_path)

for cores in range(1, 9):
    print(f"Running with {cores} core(s)")
    # Set core count for this plan
    RasPlan.set_num_cores(plan_number, cores)
    
    # Time the execution of the plan
    start_time = time.time()
    RasCmdr.compute_plan(plan_number)
    execution_time = time.time() - start_time
    
    # Store the results
    results.append({
        "plan_number": plan_number,
        "cores": cores,
        "execution_time": execution_time
    })
    
    print(f"Execution time: {execution_time:.2f} seconds")

print("Sensitivity analysis complete")

# Step 5: Convert results into a DataFrame
results_df = pd.DataFrame(results)

# Optionally, save the results to a CSV file
results_df.to_csv("core_sensitivity_results.csv", index=False)


NOTES FOR REVISIONS:
- Use HDF compute summary to show the time for each preproces/unsteady compute/postprocess step. 
- First, run preprocessor and then toggle options to only run unsteady compute and postprocess. 
- Plot each step separately. 



In [ ]:
# Optionally, load the results from a CSV file
results_df = pd.read_csv("core_sensitivity_results.csv")

# Display the results dataframe for verification
print("results_df DataFrame:")
display(results_df)

# Step 6: Calculate unit runtime (based on 1 core execution time)
results_df['unit_runtime'] = results_df.groupby('plan_number')['execution_time'].transform(lambda x: x / x.iloc[0])

# Get the project name from the ras object
project_name = ras.project_name

# Step 7: Plot a line chart for unit runtime vs. cores for each plan
plt.figure(figsize=(10, 6))
for plan in results_df['plan_number'].unique():
    plan_data = results_df[results_df['plan_number'] == plan]
    plt.plot(plan_data['cores'], plan_data['unit_runtime'], label=f"Plan {plan}")

plt.xlabel("Number of Cores")
plt.ylabel("Unit Runtime (Relative to 1 Core)")
plt.title(f"{project_name} (HEC Example Project)\nCore Count Sensitivity Analysis")
plt.legend(title="Plan Number")
plt.grid(True)
plt.show()

# Print summary statistics
print("\nSummary Statistics:")
summary_stats = results_df.groupby('cores')['execution_time'].agg(['mean', 'min', 'max'])
display(summary_stats)

# Calculate and print speedup
speedup = results_df[results_df['cores'] == 1]['execution_time'].mean() / results_df[results_df['cores'] == 8]['execution_time'].mean()
print(f"\nAverage speedup from 1 to 8 cores: {speedup:.2f}x")